In [2]:
from neural_analysis_pkg.core import NeuralAnalysis
import os 

In [3]:
# Define the path to your project folder
project_folder_path ='/home/cresp1el-local/Documents/MATLAB/Data/lmc_project_v2/LED'
# Initialize the NeuralAnalysis class with the project folder path
analysis = NeuralAnalysis(project_folder_path)

#uncomment if required
# analysis.process_dat_file(project_folder_path) #process dat file

Loaded existing recording results dataframe from /home/cresp1el-local/Documents/MATLAB/Data/lmc_project_v2/LED/SpikeStuff/recording_results.csv


In [4]:
analysis.recording_results_df #show recording results dataframe

,Unnamed: 0,group_name,recording_name,downsampled_path,rms_values,iqr,good_channels,noisy_channels,mua_data_path
0,0,Lmc_opsin,lmc_ch_1_3094_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[173.62243801281514, 172.78285340315944, 187.4...",11.150732,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
1,1,Lmc_opsin,lmc_ch_1_3094_rec2,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[168.39519181099126, 170.93270235527493, 173.7...",3.478723,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[31],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
2,2,Lmc_opsin,lmc_ch_2_3093_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[171.00985988937825, 172.24690059522268, 177.2...",6.961339,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
3,3,Lmc_noopsin,lmc_noch_1_3096_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[167.1236639222655, 168.25834381046246, 169.20...",2.053548,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...","[13, 31]",/home/cresp1el-local/Documents/MATLAB/Data/lmc...
4,4,Lmc_noopsin,lmc_noch_1_3096_rec2,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[163.65757085134805, 170.61957513200855, 171.0...",4.941244,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[],/home/cresp1el-local/Documents/MATLAB/Data/lmc...


In [ ]:
analysis.process_MUA() #process MUA

In [ ]:
# save this dataframe to a csv file using the csv file path as the argument so you dont have to run the downsampling and filtering again
# analysis.recording_results_df.to_csv(os.path.join(project_folder_path,'SpikeStuff/recording_results.csv'))

In [4]:
analysis.extract_stimulation_data() 

In [5]:
analysis.stimulation_data_df #show stimulation data dataframe

,onset_times,offset_times,stimulation_ids,group_name,recording_name
0,17.345,17.845,8,Lmc_opsin,lmc_ch_1_3094_rec1
1,18.845,19.345,8,Lmc_opsin,lmc_ch_1_3094_rec1
2,20.345,20.845,8,Lmc_opsin,lmc_ch_1_3094_rec1
3,21.845,22.345,8,Lmc_opsin,lmc_ch_1_3094_rec1
4,23.345,23.846,8,Lmc_opsin,lmc_ch_1_3094_rec1
...,...,...,...,...,...
2963,909.568,910.068,8,Lmc_noopsin,lmc_noch_1_3096_rec2
2964,911.068,911.568,8,Lmc_noopsin,lmc_noch_1_3096_rec2
2965,912.568,913.068,8,Lmc_noopsin,lmc_noch_1_3096_rec2
2966,914.068,914.568,8,Lmc_noopsin,lmc_noch_1_3096_rec2


In [ ]:
analysis.extract_spike_times() #detect spikes from MUA data using thresholding method

In [ ]:
analysis.recording_results_df


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def calculate_firing_rates(recording_name, recording_results_df, stimulation_data_df, n_channels):
    # Get the mua_data_path for the current recording to get the spike_data_path where the spike times are stored
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    # Construct the spike_data_path from the mua_data_path
    spike_data_path = mua_data_path.replace('_MUA.npy', '_spike_times.npy')
    
    # Step 1: Identify the time windows for stimulus_id = 8, which is an 8Hz signal 
    stim_data = stimulation_data_df[
        (stimulation_data_df['recording_name'] == recording_name) & 
        (stimulation_data_df['stimulation_ids'] == 8)
    ]
    
    # Step 2: Load the corresponding spike times
    spike_data = np.load(spike_data_path, allow_pickle=True)
    spike_times = spike_data['time']
    spike_channels = spike_data['channel']
    
    # Get good and noisy channels for the current recording
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]
    noisy_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'noisy_channels'
    ].values[0]

    # Step 3 & 4: Calculate and aggregate the firing rates
    firing_rates = np.full((n_channels, len(stim_data)), np.nan) # Initialize with NaNs 

    for i, (onset, offset) in enumerate(zip(stim_data['onset_times'], stim_data['offset_times'])):
        for ch in good_channels:
            # Find spikes in the current channel and time window
            condition = (spike_channels == ch) & (spike_times >= onset) & (spike_times <= offset)
            spikes_in_window = spike_times[np.where(condition)]
            
            # Calculate the firing rate
            firing_rate = len(spikes_in_window) / (offset - onset)
            firing_rates[ch, i] = firing_rate
    
    # Step 5: Plotting the heatmap
    plt.imshow(firing_rates, aspect='auto', cmap='hot', interpolation='nearest')
    plt.colorbar(label='Firing Rate (Hz)')
    plt.ylabel('Channel')
    plt.xlabel('Trial')
    plt.title('Firing Rate Heatmap for Stimulus ID = 8 Hz LED')
    plt.yticks(range(n_channels), range(1, n_channels+1))  # Label y-axis with channel numbers
    plt.show()
    
calculate_firing_rates('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels)

In [ ]:
calculate_firing_rates('lmc_ch_1_3094_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels)

In [ ]:
calculate_firing_rates('lmc_ch_1_3094_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels)

In [ ]:
calculate_firing_rates('lmc_noch_1_3096_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels)

In [ ]:
calculate_firing_rates('lmc_noch_1_3096_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels)

In [9]:
from scipy.ndimage import gaussian_filter1d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def estimate_instantaneous_firing_rate(recording_name, recording_results_df, n_channels, bin_size=0.001, window_length=0.05, window_sd=0.005):
    """
    Estimate the instantaneous firing rate by convolving the spike time series with a Gaussian window.

    Parameters:
    - recording_name (str): The name of the recording to process.
    - recording_results_df (pd.DataFrame): The data frame containing the recording results.
    - n_channels (int): The number of channels in the recording.
    - bin_size (float): The bin size for discretizing the spike times, in seconds. Default is 1 ms (0.001 s).
    - window_length (float): The length of the Gaussian window in seconds. Default is 50 ms (0.05 s).
    - window_sd (float): The standard deviation of the Gaussian window in seconds. Default is 5 ms (0.005 s).
    
    Returns:
    - firing_rate_estimates (ndarray): A 2D array where each row represents a channel and each column represents a time bin. The values represent the estimated firing rates in Hz.
    """

    # Get the mua_data_path for the current recording to get the spike_data_path where the spike times are stored
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    # Construct the spike_data_path from the mua_data_path
    spike_data_path = mua_data_path.replace('_MUA.npy', '_spike_times.npy')
    
    # Step 1: Load the spike data
    spike_data = np.load(spike_data_path, allow_pickle=True)
    spike_times = spike_data['time']
    spike_channels = spike_data['channel']

    # Step 2: Determine the duration from the MUA data
    mua_data = np.load(mua_data_path)
    duration = mua_data.shape[0] / 10000  # Convert number of samples to seconds (assuming 10 kHz sampling rate)

    # Step 3: Create a time vector with bins
    time_vector = np.arange(0, duration, bin_size)

    # Step 4: Create a spike train matrix with each row representing a channel and each column representing a time bin
    spike_trains = np.zeros((n_channels, len(time_vector) - 1))

    for ch in range(n_channels):
        spike_times_ch = spike_times[spike_channels == ch]
        spike_trains[ch, :] = np.histogram(spike_times_ch, bins=time_vector)[0]

    # Step 5: Convolve the spike train with a Gaussian window to estimate the instantaneous firing rate
    window_length_bins = int(window_length / bin_size)  # Convert window length from seconds to number of bins
    window_sd_bins = window_sd / bin_size  # Convert window SD from seconds to number of bins
    firing_rate_estimates = np.zeros_like(spike_trains)

    for ch in range(n_channels):
        firing_rate_estimates[ch, :] = gaussian_filter1d(spike_trains[ch, :], sigma=window_sd_bins)

    return firing_rate_estimates

#run the function
#firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.n_channels)

In [ ]:
# Plot the heatmap
plt.imshow(firing_rate_estimates, aspect='auto', cmap='hot', interpolation='nearest', extent=[0, firing_rate_estimates.shape[1], 0, firing_rate_estimates.shape[0]])
plt.colorbar(label='Firing Rate (Hz)')
plt.ylabel('Channel')
plt.xlabel('Time (bins)')
plt.title('Instantaneous Firing Rate')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_psth(recording_name, recording_results_df, stimulation_data_df, n_channels, firing_rate_estimates, stim_id, bin_size=0.001):
    
    # Get the mua_data_path for the current recording to get the spike_data_path where the spike times are stored
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    # Construct the spike_data_path from the mua_data_path
    spike_data_path = mua_data_path.replace('_MUA.npy', '_spike_times.npy')
    
    # Step 1: Identify the time windows for the specified stimulus_id
    stim_data = stimulation_data_df[
        (stimulation_data_df['recording_name'] == recording_name) & 
        (stimulation_data_df['stimulation_ids'] == stim_id)
    ]
    
    # Get good and noisy channels for the current recording
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]

    # Find the minimum onset and maximum offset times to get a common time window for all trials
    min_onset = stim_data['onset_times'].min() - 0.5  # 500 ms before the minimum onset
    max_offset = stim_data['offset_times'].max() + 0.5  # 500 ms after the maximum offset

    # Find the bins corresponding to the common time window
    start_bin = int(min_onset / bin_size)
    end_bin = int(max_offset / bin_size)

    # Initialize arrays to accumulate the sums and counts for each bin
    sum_psth = np.zeros((n_channels, end_bin - start_bin))
    count_psth = np.zeros((n_channels, end_bin - start_bin))

    # Loop through each trial to calculate the PSTH
    for i, (onset, offset) in enumerate(zip(stim_data['onset_times'], stim_data['offset_times'])):
        # Find the bins corresponding to the current time window
        trial_start_bin = int((onset - 0.5) / bin_size)
        trial_end_bin = int((offset + 0.5) / bin_size)

        # Add the firing rate estimates to the sum and update the count in the relevant bins
        sum_psth[:, trial_start_bin - start_bin : trial_end_bin - start_bin] += firing_rate_estimates[:, trial_start_bin:trial_end_bin]
        count_psth[:, trial_start_bin - start_bin : trial_end_bin - start_bin] += 1
    
    # Calculate the mean PSTH by dividing the sum by the count
    mean_psth = np.divide(sum_psth, count_psth, where=(count_psth!=0))

    # Plotting the mean PSTH for each channel
    time_bins = np.linspace(min_onset, max_offset, end_bin - start_bin)
    plt.figure()
    for ch in good_channels:
        plt.plot(time_bins, mean_psth[ch, :])
    
    plt.xlabel('Time (s)')
    plt.ylabel('Firing Rate (Hz)')
    plt.title(f'Mean PSTH for Stimulus ID = {stim_id}')
    plt.show()

#first get the firing rate estimates for the recording name
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.n_channels)
#run the function 
calculate_psth('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [ ]:
def calculate_psth(recording_name, recording_results_df, stimulation_data_df, n_channels, firing_rate_estimates, stim_id=1, bin_size=0.001):
    # Get the mua_data_path for the current recording
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    # Step 1: Identify the time windows for the specified stimulus_id
    stim_data = stimulation_data_df[
        (stimulation_data_df['recording_name'] == recording_name) & 
        (stimulation_data_df['stimulation_ids'] == stim_id)
    ]
    
    # Get good and noisy channels for the current recording
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]
    noisy_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'noisy_channels'
    ].values[0]

    # Exclude noisy channels from good channels
    good_channels = [ch for ch in good_channels if ch not in noisy_channels]

    # Step 2 & 3: Aggregate the PSTH data
    psth_duration_in_s = 1.5  # PSTH duration in seconds (1500 ms)
    num_bins = int(psth_duration_in_s / bin_size)
    sum_psth = np.zeros((n_channels, num_bins))  # Initialize with zeros
    count_psth = np.zeros((n_channels, num_bins))  # Initialize with zeros

    for i, (onset, offset) in enumerate(zip(stim_data['onset_times'], stim_data['offset_times'])):
        for ch in good_channels:
            # Find the bins corresponding to the current time window (from -500ms to +1000ms relative to the onset)
            start_bin = int((onset - 0.5) / bin_size)
            end_bin = int((onset + 1.0) / bin_size)
            
            # within your loop where you extract trial_psth
            trial_psth = firing_rate_estimates[ch, start_bin:end_bin][:1500]
            
            #accumulate the sum and update the count in the relevant bins
            try: 
                sum_psth[ch, :] += np.nan_to_num(trial_psth)
                count_psth[ch, :] += np.isfinite(trial_psth)
            except ValueError: 
                # If lengths are mismatched, extend trial_psth with its last value
                if len(trial_psth) == len(sum_psth[ch, :]) - 1:
                    trial_psth = np.append(trial_psth, trial_psth[-1])
                    sum_psth[ch, :] += np.nan_to_num(trial_psth)
                    count_psth[ch, :] += np.isfinite(trial_psth)
                else:
                    print("Unexpected mismatch in lengths")
 
    # Calculate the mean PSTH by dividing the sum by the count
    mean_psth = np.divide(sum_psth, count_psth, where=(count_psth!=0))
    
    # Convert firing rate from spikes per bin to spikes per second (Hz)
    mean_psth /= bin_size

    # Create a time axis that spans from -500 ms to +1000 ms
    time_axis = np.linspace(-500, 1000, num_bins)

    # Step 4: Plotting the mean PSTH for each channel
    plt.figure()
    for ch in range(n_channels):
        plt.plot(time_axis, mean_psth[ch, :])
        plt.xlabel('Time (ms)')
        plt.ylabel('Firing Rate (Hz)')
        plt.title(f'Channel {ch+1}')
        plt.axvline(x=0, color='r', linestyle='--')  # Mark stimulus onset
        plt.axvline(x=500, color='r', linestyle='--')  # Mark stimulus offset
        plt.show()

#run the function with the firing rate estimates
calculate_psth('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [ ]:
analysis.recording_results_df['recording_name']

In [ ]:
calculate_psth('lmc_ch_1_3094_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

calculate_psth('lmc_ch_1_3094_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [ ]:
calculate_psth('lmc_ch_1_3094_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [ ]:
calculate_psth('lmc_noch_1_3096_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [ ]:
calculate_psth('lmc_noch_1_3096_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

In [21]:
import numpy as np
import matplotlib.pyplot as plt

def bootstrap_ci(data, n_bootstraps=1000, ci=0.99):
    bootstrapped_means = []
    for i in range(n_bootstraps):
        random_sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(random_sample))
    lower = np.percentile(bootstrapped_means, (1-ci)/2 * 100)
    upper = np.percentile(bootstrapped_means, (1+ci)/2 * 100)
    return lower, upper


def calculate_psth_and_responsive_channels(recording_name, recording_results_df, stimulation_data_df, n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001):
    # Initialization
    prestimulus_CI = []
    poststimulus_CI = []
    responsive_channels = []
    
    # Existing data retrieval logic
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    stim_data = stimulation_data_df[
        (stimulation_data_df['recording_name'] == recording_name) & 
        (stimulation_data_df['stimulation_ids'] == stim_id)
    ]
    
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]
    noisy_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'noisy_channels'
    ].values[0]
    
    good_channels = [ch for ch in good_channels if ch not in noisy_channels]
  
    for ch in good_channels:
        all_prestim_data = []
        all_poststim_data = []
        
        for i, (onset, offset) in enumerate(zip(stim_data['onset_times'], stim_data['offset_times'])):
            prestim_start_bin = int((onset - 0.6) / bin_size)
            prestim_end_bin = int(onset / bin_size)
            
            poststim_start_bin = int(onset / bin_size)
            poststim_end_bin = int((onset + 0.1) / bin_size)
            
            # this would be pooling the data from all trials for each channel which each bin is a time point of varying voltage
            # prestim_data = firing_rate_estimates[ch, prestim_start_bin:prestim_end_bin]
            # poststim_data = firing_rate_estimates[ch, poststim_start_bin:poststim_end_bin]
            
            # use the extend method if pooling from all trials for each channel
            # all_prestim_data.extend(prestim_data)
            # all_poststim_data.extend(poststim_data)
            
            prestim_data = np.mean(firing_rate_estimates[ch, prestim_start_bin:prestim_end_bin])
            poststim_data = np.mean(firing_rate_estimates[ch, poststim_start_bin:poststim_end_bin])
            
            all_prestim_data.append(prestim_data)  # Use append here
            all_poststim_data.append(poststim_data)  # Use append here
        
        # Perform bootstrap analysis for 99% CI on the pooled data
        prestim_bootstrap = bootstrap_ci(np.array(all_prestim_data))
        poststim_bootstrap = bootstrap_ci(np.array(all_poststim_data))
        
        # Print for debugging
        print(f"Channel {ch}: Pre-stim CI: {prestim_bootstrap}, Post-stim CI: {poststim_bootstrap}")
        
        prestimulus_CI.append(prestim_bootstrap)
        poststimulus_CI.append(poststim_bootstrap)
        
        if poststim_bootstrap[0] > prestim_bootstrap[1]:
            responsive_channels.append(ch)
    print("Responsive channels:", responsive_channels)

#run the function 
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (0.022708875030623448, 0.02607367105164545), Post-stim CI: (0.022079265315967808, 0.028323988238212174)
Channel 1: Pre-stim CI: (0.026583280410268806, 0.03135554097839441), Post-stim CI: (0.024169553765334126, 0.031165518087795684)
Channel 2: Pre-stim CI: (0.04114062567857559, 0.04746479421494688), Post-stim CI: (0.03811193344814947, 0.049413870110157763)
Channel 3: Pre-stim CI: (0.03270601583896682, 0.03759625154508655), Post-stim CI: (0.03138433012562161, 0.04008075937604985)
Channel 4: Pre-stim CI: (0.09722911109616099, 0.11059080277883766), Post-stim CI: (0.09565427916565396, 0.12056142739737268)
Channel 5: Pre-stim CI: (0.0865449842545665, 0.09960689263467665), Post-stim CI: (0.08295511032837265, 0.10817946108942698)
Channel 6: Pre-stim CI: (0.05322394162739619, 0.061929760227270446), Post-stim CI: (0.051378279537035755, 0.06643988647361249)
Channel 7: Pre-stim CI: (0.04463335364130342, 0.050708215112266565), Post-stim CI: (0.04115851707385208, 0.0537668019

THIS IS THE UPDATED CODE...ned to confirm 

In [8]:
def bootstrap_ci(data, n_bootstraps=1000, ci=0.999):
    bootstrapped_means = []
    for i in range(n_bootstraps):
        random_sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(random_sample))
    lower = np.percentile(bootstrapped_means, (1-ci)/2 * 100)
    upper = np.percentile(bootstrapped_means, (1+ci)/2 * 100)
    return lower, upper

def calculate_psth_and_responsive_channels(recording_name, recording_results_df, stimulation_data_df, n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001):
    # Initialization
    prestimulus_CI = []
    poststimulus_CI = []
    responsive_channels = []
    
    avg_firing_rates_for_responsive_channels = []
    all_baseline_firing_rates = []
    
    # Existing data retrieval logic
    mua_data_path = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'mua_data_path'
    ].values[0]
    
    stim_data = stimulation_data_df[
        (stimulation_data_df['recording_name'] == recording_name) & 
        (stimulation_data_df['stimulation_ids'] == stim_id)
    ]
    
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]
    noisy_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'noisy_channels'
    ].values[0]
    
    good_channels = [ch for ch in good_channels if ch not in noisy_channels]
  
    for ch in good_channels:
        all_prestim_data = []
        all_poststim_data = []
        
        for i, (onset, offset) in enumerate(zip(stim_data['onset_times'], stim_data['offset_times'])):
            prestim_start_bin = int((onset - 0.2) / bin_size)
            prestim_end_bin = int(onset / bin_size)
            poststim_start_bin = int(onset / bin_size)
            poststim_end_bin = int((onset + 0.08) / bin_size)
            
            prestim_data = np.mean(firing_rate_estimates[ch, prestim_start_bin:prestim_end_bin]) / bin_size
            poststim_data = np.mean(firing_rate_estimates[ch, poststim_start_bin:poststim_end_bin]) / bin_size
            
            all_prestim_data.append(prestim_data)
            all_poststim_data.append(poststim_data)
        
        prestim_bootstrap = bootstrap_ci(np.array(all_prestim_data))
        poststim_bootstrap = bootstrap_ci(np.array(all_poststim_data))
        
        print(f"Channel {ch}: Pre-stim CI: {prestim_bootstrap}, Post-stim CI: {poststim_bootstrap}")
        
        prestimulus_CI.append(prestim_bootstrap)
        poststimulus_CI.append(poststim_bootstrap)
        
        if poststim_bootstrap[0] > prestim_bootstrap[1]:
            responsive_channels.append(ch)
            avg_firing_rates_for_responsive_channels.append(np.mean(all_poststim_data))
            all_baseline_firing_rates.extend(all_prestim_data)

    print("Responsive channels:", responsive_channels)
    
    single_waveform = np.mean(avg_firing_rates_for_responsive_channels)
    baseline = np.mean(all_baseline_firing_rates)
    percent_change = ((single_waveform - baseline) / baseline) * 100

    print(f"Single LED-evoked waveform: {single_waveform}")
    print(f"Baseline: {baseline}")
    print(f"Percent change relative to baseline: {percent_change}%")


In [85]:
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_ch_2_3093_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (22.930651342338066, 28.8456326905179), Post-stim CI: (21.486042665713196, 28.944805161589112)
Channel 1: Pre-stim CI: (25.531067679659895, 33.34877944423127), Post-stim CI: (23.362696321762826, 32.40066863748769)
Channel 2: Pre-stim CI: (40.28827080375227, 53.47048454861072), Post-stim CI: (36.46694890384308, 50.243858086712486)
Channel 3: Pre-stim CI: (33.25467899867823, 41.942148907100304), Post-stim CI: (30.41776484253461, 40.49641211925088)
Channel 4: Pre-stim CI: (94.5309915364878, 118.31759520609526), Post-stim CI: (88.26802024139533, 122.92865397957351)
Channel 5: Pre-stim CI: (81.77473718303794, 105.03270702642862), Post-stim CI: (73.87005227063689, 110.64144411419936)
Channel 6: Pre-stim CI: (52.2268699268011, 69.29055247524215), Post-stim CI: (49.67703898184758, 69.42839830134089)
Channel 7: Pre-stim CI: (42.3716249101685, 55.01489385376467), Post-stim CI: (39.945685996356005, 55.46725785814344)
Channel 8: Pre-stim CI: (40.27571727423711, 51.622975782

/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [86]:
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_1_3094_rec1', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_ch_1_3094_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (21.0179005163521, 30.894104207471113), Post-stim CI: (96.56463762501122, 116.36639262965043)
Channel 1: Pre-stim CI: (22.54682116130286, 27.92604279823605), Post-stim CI: (59.66428751394336, 71.99081849223094)
Channel 2: Pre-stim CI: (21.40650333232827, 31.19629732686645), Post-stim CI: (392.21617881816826, 415.4453013588848)
Channel 3: Pre-stim CI: (20.72412505075317, 29.59096353980612), Post-stim CI: (104.62013184102534, 118.76054059226153)
Channel 4: Pre-stim CI: (30.717636907073953, 42.44888751797531), Post-stim CI: (150.9791984068319, 170.70598177271808)
Channel 5: Pre-stim CI: (21.422650529888006, 29.365339075450457), Post-stim CI: (79.57630922367501, 94.5740887486953)
Channel 6: Pre-stim CI: (20.017921309473692, 30.98630017764898), Post-stim CI: (184.22658353656354, 206.9037870500104)
Channel 7: Pre-stim CI: (30.367978529617705, 46.54045474622011), Post-stim CI: (544.7761870457, 572.075047878979)
Channel 8: Pre-stim CI: (22.89017464338132, 29.65078911657

In [87]:
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_ch_1_3094_rec2', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_ch_1_3094_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (12.688069108213444, 15.493967931967475), Post-stim CI: (26.93392863606817, 33.691746146206846)
Channel 1: Pre-stim CI: (16.83796953943861, 21.225590746683523), Post-stim CI: (20.40236202526182, 27.515778447039033)
Channel 2: Pre-stim CI: (16.178252299713794, 21.45329330319549), Post-stim CI: (38.62885825542416, 49.058759869294875)
Channel 3: Pre-stim CI: (13.74355121869015, 17.109062433370998), Post-stim CI: (17.455733968603457, 22.795000770098763)
Channel 4: Pre-stim CI: (25.17432927481371, 36.113876311370326), Post-stim CI: (129.5735529790152, 145.12842915059417)
Channel 5: Pre-stim CI: (24.358286255226396, 34.0915026686134), Post-stim CI: (119.7557334811307, 135.89148398134978)
Channel 6: Pre-stim CI: (14.638870787549807, 17.630999571859054), Post-stim CI: (22.7858008213008, 28.233355662235773)
Channel 7: Pre-stim CI: (16.156674090047293, 21.734323104242154), Post-stim CI: (73.7008777792474, 83.02445141145408)
Channel 8: Pre-stim CI: (16.42750458167206, 20.1

In [88]:
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_noch_1_3096_rec1', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_noch_1_3096_rec1', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (15.908621123622623, 19.328259348023938), Post-stim CI: (16.11412905128388, 20.49884707204594)
Channel 1: Pre-stim CI: (18.33959589559744, 25.685980681313858), Post-stim CI: (17.80609391317706, 29.307687250751886)
Channel 2: Pre-stim CI: (16.08718751011653, 23.614597843687005), Post-stim CI: (16.28854623222757, 27.552994397852924)
Channel 3: Pre-stim CI: (16.759233434768387, 22.88991799546297), Post-stim CI: (14.33088237343865, 24.170845621411992)
Channel 4: Pre-stim CI: (15.277839173946152, 19.548982151851945), Post-stim CI: (17.759864321016316, 27.050047811624882)
Channel 5: Pre-stim CI: (17.33727087171951, 22.25873241499033), Post-stim CI: (18.83179576794995, 28.325922315837026)
Channel 6: Pre-stim CI: (15.51954687397986, 23.21577342200861), Post-stim CI: (14.827949063871324, 23.964328954323857)
Channel 7: Pre-stim CI: (21.116460474454925, 27.126362408586306), Post-stim CI: (19.284370641297805, 29.501573014092582)
Channel 8: Pre-stim CI: (20.77523337703592, 2

In [89]:
firing_rate_estimates = estimate_instantaneous_firing_rate('lmc_noch_1_3096_rec2', analysis.recording_results_df, analysis.n_channels)
calculate_psth_and_responsive_channels('lmc_noch_1_3096_rec2', analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, firing_rate_estimates, stim_id=8, bin_size=0.001)

Channel 0: Pre-stim CI: (17.294725935364294, 19.600630899081764), Post-stim CI: (17.844955777264733, 21.680839808747074)
Channel 1: Pre-stim CI: (20.53453610915094, 32.038227689319015), Post-stim CI: (21.75021012648889, 39.1337586135728)
Channel 2: Pre-stim CI: (13.637944101177757, 16.396281753607205), Post-stim CI: (12.148741377893376, 15.654412083050616)
Channel 3: Pre-stim CI: (13.341643635327415, 15.697270858840415), Post-stim CI: (13.872539193350926, 17.785241527019075)
Channel 4: Pre-stim CI: (16.79327129635381, 21.598701711416076), Post-stim CI: (16.275712585592853, 23.27675815151795)
Channel 5: Pre-stim CI: (17.842179969494648, 24.100434678042337), Post-stim CI: (17.81765592589324, 25.95023232638654)
Channel 6: Pre-stim CI: (13.843176860276882, 16.379701637976176), Post-stim CI: (14.058058744204706, 18.162000993754344)
Channel 7: Pre-stim CI: (14.200720530133712, 17.689809348700596), Post-stim CI: (14.349814743919165, 18.650320949005287)
Channel 8: Pre-stim CI: (19.554460493205

In [ ]:
analysis.recording_results_df['recording_name']

In [ ]:
analysis.recording_results_df['recording_name'][2]

In [ ]:
analysis.recording_results_df

In [13]:
def calculate_psth_pre_post(recording_name, recording_results_df, stimulation_data_df, n_channels, firing_rate_estimates, bin_size=0.001, pre_trials=30, post_trials=30):
    # Get good and noisy channels for the current recording
    good_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'good_channels'
    ].values[0]
    noisy_channels = recording_results_df.loc[
        recording_results_df['recording_name'] == recording_name, 
        'noisy_channels'
    ].values[0]

    good_channels = list(set(good_channels) - set(noisy_channels))

    for ch in good_channels:  # Loop through each channel
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))  # Create a 1x4 subplot for each channel

        for stim_id in range(1, 5):  # Loop through each stimulation ID
            ax = axs[stim_id - 1]  # Get the correct axes

            # Separate the data into pre and post epochs based on the trial range specified
            stim_data = stimulation_data_df[
                (stimulation_data_df['recording_name'] == recording_name) & 
                (stimulation_data_df['stimulation_ids'] == stim_id)
            ]
            stim_data_pre = stim_data.iloc[:pre_trials] # grabs all the rows up to the pre_trials value 
            stim_data_post = stim_data.iloc[-post_trials:] # grabs all the rows from the end of the dataframe to the post_trials value

            # Calculate and plot the mean PSTH for the pre epoch
            mean_psth_pre = calculate_mean_psth(stim_data_pre, firing_rate_estimates, ch, bin_size)
            ax.plot(mean_psth_pre, color='grey', label='Pre')

            # Calculate and plot the mean PSTH for the post epoch
            mean_psth_post = calculate_mean_psth(stim_data_post, firing_rate_estimates, ch, bin_size)
            ax.plot(mean_psth_post, color='blue', label='Post')

            ax.set_title(f'Stim ID = {stim_id}')
            ax.legend()

        plt.tight_layout()
        plt.show()

def calculate_mean_psth(stim_data, firing_rate_estimates, ch, bin_size):
    psth_data = []
    for i, onset in enumerate(stim_data['onset_times']):
        # Define a time window of 1500ms centered on the stimulus onset (500ms pre-stimulus to 1000ms post-stimulus)
        start_bin = int((onset - 0.5) / bin_size)
        end_bin = int((onset + 1.0) / bin_size)
        
        # Get the PSTH data for the current trial
        trial_psth = firing_rate_estimates[ch, start_bin:end_bin]
        psth_data.append(trial_psth)
    
    # Ensuring all trials have the same shape by padding with NaNs to the maximum trial length
    max_len = max(map(len, psth_data))
    psth_data = [np.pad(trial, (0, max_len - len(trial)), 'constant', constant_values=np.nan) for trial in psth_data]

    # Calculate the mean PSTH across trials
    mean_psth = np.nanmean(np.stack(psth_data), axis=0)
        
    # Convert firing rate from spikes per bin to spikes per second (Hz)
    mean_psth /= bin_size
    
    return mean_psth

In [5]:
# Define the path to your project folder
project_folder_path ='/home/cresp1el-local/Documents/MATLAB/Data/lmc_project_v2/Whisker'
# Initialize the NeuralAnalysis class with the project folder path
analysis = NeuralAnalysis(project_folder_path)
analysis.extract_stimulation_data() 


Loaded existing recording results dataframe from /home/cresp1el-local/Documents/MATLAB/Data/lmc_project_v2/Whisker/SpikeStuff/recording_results.csv


In [6]:
analysis.recording_results_df

,Unnamed: 0,group_name,recording_name,downsampled_path,rms_values,iqr,good_channels,noisy_channels,mua_data_path
0,0,Lmc_opsin,lmc_ch_2_3093_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[170.52816270803297, 172.8074476873955, 175.11...",4.686566,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...",[13],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
1,1,Lmc_opsin,lmc_ch_1_3094_rec2,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[169.76160010287387, 172.07686648663235, 174.7...",3.035201,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[31],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
2,2,Lmc_opsin,lmc_ch_1_3094_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[174.26712509310488, 172.37445379634585, 180.8...",7.463998,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
3,3,Lmc_noopsin,lmc_noch_1_3096_rec1,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[169.26424775999243, 171.4846387576975, 173.33...",2.762267,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[31],/home/cresp1el-local/Documents/MATLAB/Data/lmc...
4,4,Lmc_noopsin,lmc_noch_1_3096_rec2,/home/cresp1el-local/Documents/MATLAB/Data/lmc...,"[170.05101760301815, 172.9588068684812, 170.34...",3.586824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",[31],/home/cresp1el-local/Documents/MATLAB/Data/lmc...


In [7]:
analysis.stimulation_data_df

,onset_times,offset_times,stimulation_ids,group_name,recording_name
0,19.523,20.023,4,Lmc_opsin,lmc_ch_2_3093_rec1
1,27.202,27.702,4,Lmc_opsin,lmc_ch_2_3093_rec1
2,39.451,39.951,3,Lmc_opsin,lmc_ch_2_3093_rec1
3,41.759,42.259,2,Lmc_opsin,lmc_ch_2_3093_rec1
4,50.476,50.976,1,Lmc_opsin,lmc_ch_2_3093_rec1
...,...,...,...,...,...
4966,7972.475,7972.975,2,Lmc_noopsin,lmc_noch_1_3096_rec2
4967,7982.583,7983.083,1,Lmc_noopsin,lmc_noch_1_3096_rec2
4968,7988.389,7988.889,4,Lmc_noopsin,lmc_noch_1_3096_rec2
4969,7994.956,7995.456,3,Lmc_noopsin,lmc_noch_1_3096_rec2


In [ ]:
def calculate_luciferin_responsive_electrodes(recording_results_df, stimulation_data_df, n_channels, bin_size=0.001):
    # Define the timestamps for when luciferin was added for each recording
    luciferin_timestamps = {
        'lmc_ch_1_3094_rec1': 4.076459533333333e+03,
        'lmc_ch_1_3094_rec2': 4.082218733333333e+03,
        'lmc_ch_2_3093_rec1': 4.076533433333334e+03,
        'lmc_noch_1_3096_rec1': 4.080881200000000e+03,
        'lmc_noch_1_3096_rec2': 4.088738733333333e+03,
    }
    
    # Iterate over unique recording names
    for recording_name in recording_results_df['recording_name'].unique():
        
        print(f"Estimating firing rates for {recording_name}...")
        # Get the firing rate estimates for the current recording using the estimate_instantaneous_firing_rate function
        firing_rate_estimates = estimate_instantaneous_firing_rate(recording_name, recording_results_df, n_channels)
        
        print(f"Analyzing {recording_name}...")
        luciferin_responsive_channels = []

        # Existing data retrieval logic
        good_channels = recording_results_df.loc[
            recording_results_df['recording_name'] == recording_name, 
            'good_channels'
        ].values[0]
        noisy_channels = recording_results_df.loc[
            recording_results_df['recording_name'] == recording_name, 
            'noisy_channels'
        ].values[0]
        good_channels = [ch for ch in good_channels if ch not in noisy_channels]

        # Fetch the time when luciferin was added for the current recording
        luciferin_time = luciferin_timestamps.get(recording_name, None)
        if luciferin_time is None:
            print(f"No luciferin timestamp found for {recording_name}. Skipping.")
            continue

        for ch in good_channels:
            all_baseline_data = []
            all_luciferin_data = []

            # Filter data for stimulations with ID == 1
            stim_data_all = stimulation_data_df[
                (stimulation_data_df['recording_name'] == recording_name) &
                (stimulation_data_df['stimulation_ids'] == 1)
            ]

            # Filter pre- and post-luciferin data
            stim_data_pre = stim_data_all[stim_data_all['onset_times'] < luciferin_time]
            stim_data_post = stim_data_all[stim_data_all['onset_times'] > luciferin_time]
            
            # Collect data for baseline (pre-luciferin)
            for onset in stim_data_pre['onset_times']:
                start_bin = int(onset / bin_size)
                end_bin = int((onset + 0.5) / bin_size)
                
                # print(f"Start bin for baseline: {start_bin}, End bin for baseline: {end_bin}")
                baseline_data = np.mean(firing_rate_estimates[ch, start_bin:end_bin])
                all_baseline_data.append(baseline_data)

            # Collect data for post-luciferin
            for onset in stim_data_post['onset_times']:
                start_bin = int(onset / bin_size)
                end_bin = int((onset+0.5)/ bin_size)
                # print(f"Start bin for post-luciferin: {start_bin}, End bin for post-luciferin: {end_bin}")
                luciferin_data = np.mean(firing_rate_estimates[ch, start_bin:end_bin])
                all_luciferin_data.append(luciferin_data)
            
            # Perform bootstrap analysis for 99.9% CI
            baseline_bootstrap = bootstrap_ci(np.array(all_baseline_data), ci=0.99)
            luciferin_bootstrap = bootstrap_ci(np.array(all_luciferin_data), ci=0.99)

            # Check if luciferin-responsive
            if luciferin_bootstrap[0] > baseline_bootstrap[1]:
                luciferin_responsive_channels.append(ch)

        print(f"Luciferin-responsive channels for {recording_name}: {luciferin_responsive_channels}")

# Call the function
calculate_luciferin_responsive_electrodes(analysis.recording_results_df, analysis.stimulation_data_df, analysis.n_channels, bin_size=0.001)



Estimating firing rates for lmc_ch_2_3093_rec1...
Analyzing lmc_ch_2_3093_rec1...


/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/cresp1el-local/.conda/envs/neural_analysis_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, 